![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [23]:
import pandas as pd
import numpy as np
# Read in bank_marketing.csv as a pandas DataFrame.
marketing=pd.read_csv("bank_marketing.csv")
marketing.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,13,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,19,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,23,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,27,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,3,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [24]:
marketing.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx',
       'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')

In [25]:
# spliting the dataframe of bank_markiting  into client,campaign ,economics with the guied what data frame  should contain which type of coloumn_value
client = marketing[["client_id", "age", "job", "marital", "education", 
             "credit_default", "housing", "loan"]]
campaign = marketing[["client_id", "campaign", "month", "day", 
               "duration", "pdays", "previous", "poutcome", "y"]]
economics = marketing[["client_id", "emp_var_rate", "cons_price_idx", 
                "euribor3m", "nr_employed"]]



In [26]:
# renaming the column name
client.rename(columns={'client_id':'id'},inplace=True)
campaign.rename(columns={'duration':'contact_duration','previous':"previous_campaign_contacts","y":"campaign_outcome","poutcome":"previous_outcome","campaign":"number_contacts"},inplace=True)
economics.rename(columns={"euribor3m":"euribor_three_months","nr_employed": "number_employed" },inplace=True)

In [27]:
client["education"].isnull().sum()

0

In [28]:
economics.head()

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


In [29]:
campaign['previous_outcome'].unique()


array(['nonexistent', 'failure', 'success'], dtype=object)

In [30]:
campaign['campaign_outcome'].unique()

array(['no', 'yes'], dtype=object)

In [31]:
# data preprocessing 
#cleaning client coloumn
for index ,row in client.iterrows():
    client.at[index,"education"]=row["education"].replace(".","_")
    if row["education"]=="unknown":
        client.at[index,"education"]=np.nan
#removing periods (".") drom the column job
client["job"]=client["job"].str.replace(".","")


In [32]:
# Convert "success" and "failure" in the "previous_outcome" and "campaign_outcome" columns to binary (1 or 0)

for index,row in campaign.iterrows():
# cleaning and manuplating the coloumn "previous_outcome'"
    if row['previous_outcome']=='nonexistent':
        campaign.at[index,'previous_outcome']=np.nan
    elif  row['previous_outcome']=='failure':
        campaign.at[index,'previous_outcome']=row['previous_outcome'].replace("failure",'0')
    else :
        campaign.at[index,'previous_outcome']=row['previous_outcome'].replace("success",'1')
# cleaning and manuplating the coloumn "campaign_outcome"      
    if row['campaign_outcome']=='yes':
         campaign.at[index,'campaign_outcome']=row['campaign_outcome'].replace("yes",'1')
    else:
         campaign.at[index,'campaign_outcome']=row['campaign_outcome'].replace("no",'0')

In [33]:
#Add a column called campaign_id in campaign, where all rows have a value of 1.
campaign['campaign_id']= 1

In [34]:
marketing[["month" , "day"]].head()

,month,day
0,may,13
1,may,19
2,may,23
3,may,27
4,may,3


In [35]:
# Create the "last_contact_date" column in df_campaing
campaign['last_contact_date'] = pd.to_datetime( "2022-" +
   marketing["month"].astype(str) + "-" + marketing["day"].astype(str) 
)

# Split the "last_contact_date" column into date and time columns
campaign[['date', 'time']] = campaign['last_contact_date'].dt.strftime('%Y-%m-%d %H:%M:%S').str.split(" ", expand=True)

In [36]:
# droing the extra columns
campaign=campaign.drop(['time','last_contact_date'],axis=1)

In [37]:
# rename the cloumns with the correct name 
campaign=campaign.rename(columns={'date':'last_contact_date'})

In [38]:
# converting the type of column into date and time 
campaign['last_contact_date']=pd.to_datetime(campaign['last_contact_date'])

In [39]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   month                       41188 non-null  object        
 3   day                         41188 non-null  int64         
 4   contact_duration            41188 non-null  int64         
 5   pdays                       41188 non-null  int64         
 6   previous_campaign_contacts  41188 non-null  int64         
 7   previous_outcome            5625 non-null   object        
 8   campaign_outcome            41188 non-null  object        
 9   campaign_id                 41188 non-null  int64         
 10  last_contact_date           41188 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(7), object(3)
memory usage

In [40]:
# saving dataframe of client,campaign,economics as CSV. file
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)
print(client.columns)

Index(['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan'],
      dtype='object')


In [41]:
# Create the client table and specify the SQL code as a multi-line string
client_table = """
CREATE TABLE client (
    id SERIAL PRIMARY KEY,
    age INT,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
-- Copy data from client.csv into the client table
\copy client FROM 'client.csv' DELIMITER ',' CSV HEADER;
"""

# Create the campaign table and specify SQL code as a multi-line string
campaign_table = """
CREATE TABLE campaign (
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL REFERENCES client (id),
    number_contacts INT,
    contact_duration INT,
    pdays INT,
    previous_campaign_contacts INT,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE
);
-- Copy data from campaign.csv into the campaign table
\copy campaign FROM 'campaign.csv' DELIMETER ',' CSV HEADER;
"""

# create the economics table and specify SQL code as a multi-line string
economics_table = """
CREATE TABLE economics (
    client_id SERIAL REFERENCES client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
-- Copy data from economics.csv into the economics table
\copy economics FROM 'economics.csv' DELIMETER ',' CSV HEADER;
"""